In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import copy
import torch.optim as optim


NUM_ATTRIBUTES = 78

In [ ]:
# data = pd.concat(map(pd.read_csv, ["Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", 
#                                  "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
#                                  "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
#                                  "Friday-WorkingHours-Morning.pcap_ISCX.csv",
#                                  "Monday-WorkingHours.pcap_ISCX.csv",
#                                  "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
#                                  "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
#                                  "Tuesday-WorkingHours.pcap_ISCX.csv",
#                                  "Wednesday-workingHours.pcap_ISCX.csv"]))
data = pd.read_csv("Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv", header=None)

X = data.iloc[:, 0:NUM_ATTRIBUTES] # obtain all instances and there columns except the last column
y = data.iloc[:, NUM_ATTRIBUTES:] # obtain all instances and only the last column (label)

X = X.drop(0) #gets rid of the first row (features names)
y = y.drop(0) #gets rid of the first row (label name - Label)
print("TOTAL ROWS PARSED: ", len(X.values)) # prints the total num of instances 

X_VALUES = (X.values).astype('float32') # converts all numbers into floating point values 32

X_VALUES[np.isnan(X_VALUES)] = 0 #replaces all values of nan with 0
X_VALUES[np.isinf(X_VALUES)] = 0 #replaces all values of inf with 0

np.savetxt('test1.txt', X_VALUES, fmt='%d') 

In [ ]:
# Transforms the y outputs to be encoded
# encode all the possible labels an instance can have
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit(y)
print(ohe.categories_)
 
# transform the y labels to be a digestible list 
# ex.) ['BENIGN', 'PortScan'] => [1. 0.] (means the instance has the label of BENIGN)
y = ohe.transform(y)
print(y)
print(len(y[0]))
output_size = len(y[0])





In [ ]:
# architecture of the network
class Multiclass(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_one = nn.Linear(78, 154) # seventy eight for all the input features and one hundred and fifty four for the expansion
        self.hidden_two = nn.Linear(154, 78) # another network layer
        self.act = nn.ReLU()
        self.output = nn.Linear(78, output_size) # output size is the appropriate y label
        
    def forward(self, x):
        x = self.act(self.hidden_one(x)) 
        x = self.act(self.hidden_two(x))
        x = self.output(x)
        return x
    
model = Multiclass()

Below is how you can define the loss metric. The CrossEntropyLoss function in PyTorch combines the softmax function with the cross entropy calculation, so you don’t need any activation function at the output layer of your model. You also need an optimizer, and Adam is chosen below.

In [ ]:
# obtain the loss function we will use (used to determine how good our model preforms)
loss_fn = nn.CrossEntropyLoss()
# optimizer used to improve our model
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:

import tqdm
 
# convert pandas DataFrame features (X) and numpy array (y) into PyTorch tensors
X = torch.tensor(X_VALUES, dtype=torch.float32) # convert feature values into tensors
y = torch.tensor(y, dtype=torch.float32) # convert label values in tensors
 
#split the data into x training data (80% of feature data), x test data (20% of feature data), y training data (80% of label data),  y test data (20% of label data)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True) 

# prepare model and training parameters
n_epochs = 10 # number of times we iterate over the data
batch_size = 100 # number of samples processed before an update is preformed
batches_per_epoch = len(X_train) // batch_size
 
best_acc = - np.inf   # init to negative infinity
best_weights = None
train_loss_hist = []
train_acc_hist = []
test_loss_hist = []
test_acc_hist = []
 
# training loop
for epoch in range(n_epochs):
    epoch_loss = [] # keeps track of the overall loss
    epoch_acc = [] # keeps track of the accuracy after each iteration on the training data
    # set model in training mode and run through each batch
    model.train()
    with tqdm.trange(batches_per_epoch, unit="batch", mininterval=0) as bar:
        bar.set_description(f"Epoch {epoch}")
        for i in bar:
            # take a batch
            start = i * batch_size
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            
            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # compute and store metrics
            acc = (torch.argmax(y_pred, 1) == torch.argmax(y_batch, 1)).float().mean()
            epoch_loss.append(float(loss))
            epoch_acc.append(float(acc))
            bar.set_postfix(
                loss=float(loss),
                acc=float(acc)
            )
    # set model in evaluation mode and run through the test set
    model.eval()
    y_pred = model(X_test)
    ce = loss_fn(y_pred, y_test)
    acc = (torch.argmax(y_pred, 1) == torch.argmax(y_test, 1)).float().mean()
    ce = float(ce)
    acc = float(acc)
    train_loss_hist.append(np.mean(epoch_loss))
    train_acc_hist.append(np.mean(epoch_acc))
    test_loss_hist.append(ce)
    test_acc_hist.append(acc)
    if acc > best_acc:
        best_acc = acc
        best_weights = copy.deepcopy(model.state_dict())
    print(f"Epoch {epoch} validation: Cross-entropy={ce:.2f}, Accuracy={acc*100:.1f}%")
 
# Restore best model
model.load_state_dict(best_weights)
 
# Plot the loss and accuracy
plt.plot(train_loss_hist, label="train")
plt.plot(test_loss_hist, label="test")
plt.xlabel("epochs")
plt.ylabel("cross entropy")
plt.legend()
plt.show()
 
plt.plot(train_acc_hist, label="train")
plt.plot(test_acc_hist, label="test")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show()

In [ ]:
# saves the model to PATH variable
PATH = './test_model.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
PATH = './test_model.pth'
model = Multiclass() # initialize your model class
model.load_state_dict(torch.load(PATH))

# convert pandas DataFrame features (X) and numpy array (y) into PyTorch tensors
X = torch.tensor(X_VALUES, dtype=torch.float32) # convert feature values into tensors
y = torch.tensor(y, dtype=torch.float32) # convert label values in tensors
 
#split the data into x training data (80% of feature data), x test data (20% of feature data), y training data (80% of label data),  y test data (20% of label data)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True)


attack_test_count = 0
benign_test_count = 0

attack_array = []
index_array = []

# obtains the malicious instances
for index, label in enumerate(y_test):
    benign_test_count += 1
    if False in torch.eq(torch.tensor([1., 0.]), label):
        index_array.append(index)
        attack_test_count += 1

print(index_array)
# loops through each malicious instance and tests it
for index in index_array:
    y_pred = model(X_test[index])
    ce = loss_fn(y_pred, y_test[index])
    # acc = (torch.argmax(y_pred) == torch.argmax(y_test[index])).float().mean()
    # ce = float(ce)
    # acc = float(acc)
    print(torch.argmax(y_pred) == torch.argmax(y_test[index]))
    print(torch.argmax(y_pred))
    print(torch.argmax(y_test[index]))
    


